In [9]:
#Module Imports
import numpy as np
import numpy.random as npr
import math
from pylab import *
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats
import time

In [10]:
'''DEFINITION OF VARIABLES
    S0 - Stock Price at T=0
    E - Strike Price
    T - Time in Years
    R - Risk Free Rate
    SIGMA - Volatility
    DT - Time Step = T/N
    DF - Discount Factor = e^-RT
    I - Number of Simulations
    P - Discrete Sampling Frequency for Asian Options 
        252/Annual, 126/SemiAnnual, 63/Quarterly, 21/Monthly, 1/Continuous
'''

S0 = 100
E=100
T=1
R=0.05
SIGMA=0.20
I=100000
P= 21 #Discrete Sampling Frequency 252/Annual, 126/SemiAnnual, 63/Quarterly, 21/Monthly, 1/Continuous
N=252

In [11]:
'''OPTION VALUATION - MONTE CARLO SIMULATION W/ ANTITHETIC VARIANCE REDUCTION W/ MILSTEIN SCHEME '''
def option_valuation_Milstein(S0, E, T, N, SIGMA, R, I, P):    
    DT = T/N   #Time Step   
#GENERATE RANDOM NUMBERS - ANTITHETIC VARIANCE REDUCTION
    PHI = npr.standard_normal((N,int(I/2))) 
    PHI = np.concatenate((PHI, -PHI), axis=1)     
#SET UP EMPTY ARRAYS AND SET INITIAL VALUES    
    S = np.zeros_like(PHI)  #Array to Capture Asset Value Path
    S[0] = S0
    
#CREATE FOR LOOP TO GENERATE SIMULATION PATHS - MILSTEIN METHOD
    for t in range (1,N):
        S[t]=S[t-1]*(1+R*DT+(SIGMA*PHI[t]*np.sqrt(DT))+(SIGMA**2)*(0.5*(((PHI[t]**2)-1)*DT)))
    
    return S

In [12]:
%%timeit -n1
#Calculation of Discounted Expected Payoff
S = option_valuation_Milstein(S0, E, T, N, SIGMA, R, I, P)

DF = math.exp(-R*T)  #Discount Factor

print("Number of Simulations %.d" %I)

Call_Value = DF * np.sum(np.maximum(S[-1] - E, 0)) / I
print( "Value of Call Option Monte Carlo W/ Milstein = %.3f" %Call_Value)
Put_Value = DF * np.sum(np.maximum(E - S[-1], 0)) / I
print( "Value of Put Option Monte Carlo W/ Milstein = %.3f" %Put_Value)

Number of Simulations 100000
Value of Call Option Monte Carlo W/ Milstein = 10.422
Value of Put Option Monte Carlo W/ Milstein = 5.567
Number of Simulations 100000
Value of Call Option Monte Carlo W/ Milstein = 10.387
Value of Put Option Monte Carlo W/ Milstein = 5.546
Number of Simulations 100000
Value of Call Option Monte Carlo W/ Milstein = 10.341
Value of Put Option Monte Carlo W/ Milstein = 5.515
1 loop, best of 3: 1.51 s per loop


In [13]:
'''OPTION VALUATION - MONTE CARLO SIMULATION W/ ANTITHETIC VARIANCE REDUCTION W/O MILSTEIN SCHEME '''
def option_valuation(S0, E, T, N, SIGMA, R, I, P):    
    DT = T/N   #Time Step   
#GENERATE RANDOM NUMBERS - ANTITHETIC VARIANCE REDUCTION
    PHI = npr.standard_normal((N,int(I/2))) 
    PHI = np.concatenate((PHI, -PHI), axis=1)     
#SET UP EMPTY ARRAYS AND SET INITIAL VALUES    
    S = np.zeros_like(PHI)  #Array to Capture Asset Value Path
    S[0] = S0
    
#CREATE FOR LOOP TO GENERATE SIMULATION PATHS - EULER-MARUYAMA METHOD
    for t in range (1,N):
        S[t]=S[t-1]*(1+R*DT+(SIGMA*PHI[t]*np.sqrt(DT)))
    
    return S

In [14]:
%%timeit -n1
#Calculation of Discounted Expected Payoff
S = option_valuation(S0, E, T, N, SIGMA, R, I, P)

DF = math.exp(-R*T)  #Discount Factor

print("Number of Simulations %.d" %I)

Call_Value = DF * np.sum(np.maximum(S[-1] - E, 0)) / I
print( "Value of Call Option Euler-Maruyama Method Monte Carlo = %.3f" %Call_Value)
Put_Value = DF * np.sum(np.maximum(E - S[-1], 0)) / I
print( "Value of Put Option Euler-Maruyama Method Monte Carlo = %.3f" %Put_Value)

Number of Simulations 100000
Value of Call Option Euler-Maruyama Method Monte Carlo = 10.407
Value of Put Option Euler-Maruyama Method Monte Carlo = 5.563
Number of Simulations 100000
Value of Call Option Euler-Maruyama Method Monte Carlo = 10.447
Value of Put Option Euler-Maruyama Method Monte Carlo = 5.579
Number of Simulations 100000
Value of Call Option Euler-Maruyama Method Monte Carlo = 10.456
Value of Put Option Euler-Maruyama Method Monte Carlo = 5.583
1 loop, best of 3: 1.08 s per loop
